<a href="https://colab.research.google.com/github/NyusenKyusai/18018535_DataAnalytics/blob/main/TestingDNNEncoded.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here I will creating a model for encoded DNN with all outliers removed.

In [1]:
# needed to create the data frame
import pandas as pd

# create data frame from csv file we hosted on our github
df1 = pd.read_csv('https://raw.githubusercontent.com/NyusenKyusai/18018535_DataAnalytics/main/dnnEncodedTDWeak.csv', index_col=0)
df2 = pd.read_csv('https://raw.githubusercontent.com/NyusenKyusai/18018535_DataAnalytics/main/dnnEncodedTDSnowWeak.csv', index_col=0)
df3 = pd.read_csv('https://raw.githubusercontent.com/NyusenKyusai/18018535_DataAnalytics/main/dnnEncodedTDMaxMinWeak.csv', index_col=0)
df4 = pd.read_csv('https://raw.githubusercontent.com/NyusenKyusai/18018535_DataAnalytics/main/dnnEncodedTDMaxMinSndpWeak.csv', index_col=0)

In [2]:
# make sure we have our data by printing it out
print(df1[:6])

   Apr  Aug  Dec  Feb  Jan  Jul  Jun  Mar  May  Nov  Oct  Sep  year  temp  \
1    0    0    0    0    1    0    0    0    0    0    0    0  2015  25.7   
2    0    0    0    0    1    0    0    0    0    0    0    0  2015  31.5   
3    0    0    0    0    1    0    0    0    0    0    0    0  2015  29.0   
4    0    0    0    0    1    0    0    0    0    0    0    0  2015   2.2   
5    0    0    0    0    1    0    0    0    0    0    0    0  2015   8.1   
6    0    0    0    0    1    0    0    0    0    0    0    0  2015   4.8   

   dewp  NUM_CRIMES  
1  16.0         674  
2  27.0         648  
3  26.1         514  
4  -7.4         523  
5  -0.5         590  
6  -3.9         647  


In [152]:
# needed to help with speedy maths based calculations
import numpy as np

# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df4.iloc[np.random.permutation(len(df4))]

# df1 = 15
# df2 = 16
# df3 = 17
# df4 = 18
# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:18]

# print out the first 6 rows of predictors.
print(predictors[:6])

      Apr  Aug  Dec  Feb  Jan  Jul  Jun  Mar  May  Nov  Oct  Sep  year  temp  \
1339    0    0    0    0    0    0    0    0    0    0    0    1  2018  56.6   
1497    0    0    0    0    0    0    0    1    0    0    0    0  2019  44.9   
738     0    0    0    0    1    0    0    0    0    0    0    0  2017  27.8   
347     0    0    1    0    0    0    0    0    0    0    0    0  2015  38.4   
271     0    0    0    0    0    0    0    0    0    0    1    0  2015  53.2   
1019    0    0    0    0    0    0    0    0    0    1    0    0  2017  41.0   

      dewp   max   min  sndp  
1339  49.4  62.1  42.1   0.0  
1497  36.8  57.9  24.3   0.0  
738   20.6  34.0  24.1   0.0  
347   36.0  42.8  35.1   0.0  
271   43.7  64.9  42.1   0.0  
1019  30.6  52.0  32.0   0.0  


In [153]:
# print out the shuffled data (first 5 rows)
shuffle[:5]

,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep,year,temp,dewp,max,min,sndp,NUM_CRIMES
1339,0,0,0,0,0,0,0,0,0,0,0,1,2018,56.6,49.4,62.1,42.1,0.0,741
1497,0,0,0,0,0,0,0,1,0,0,0,0,2019,44.9,36.8,57.9,24.3,0.0,688
738,0,0,0,0,1,0,0,0,0,0,0,0,2017,27.8,20.6,34.0,24.1,0.0,784
347,0,0,1,0,0,0,0,0,0,0,0,0,2015,38.4,36.0,42.8,35.1,0.0,627
271,0,0,0,0,0,0,0,0,0,0,1,0,2015,53.2,43.7,64.9,42.1,0.0,729


In [154]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

1339    741
1497    688
738     784
347     627
271     729
1019    717
Name: NUM_CRIMES, dtype: int64


In [155]:
# A scale is not required here, but the constant will be useful in the assignment.
SCALE_NUM_TRIPS = 1.0

In [156]:
# Split our data into a training set i.e. 75% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_CRIMES'])*0.75)
# Split our data into a test set i.e. 20% of the length of the shuffle array
testsize = int(len(shuffle['NUM_CRIMES'])*0.2)
# The final test set size is 100% - 75% - 20% = 5% of the length of the shuffle array.
finaltestsize = len(shuffle['NUM_CRIMES']) - trainsize - testsize

# Define the number of input values (predictors)
# This needs to change with the dataset
nppredictors = 18
# Define the number of output values (targets)
noutputs = 1

In [157]:
# import tensorflow
%tensorflow_version 1.x
import tensorflow as tf

# check the version
print(tf.__version__)

# needed for high-level file management
import shutil  

# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
# Different Data sets
# dnnEncodedTDWeak
# dnnEncodedTDSnowWeak
# dnnEncodedTDMaxMinWeak
# dnnEncodedTDMaxMinSndpWeak
shutil.rmtree('/tmp/dnnEncodedTDMaxMinSndpWeak5', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/dnnEncodedTDMaxMinSndpWeak5', hidden_units=[20,18,14], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_TRIPS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:testsize + trainsize].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_NUM_TRIPS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:testsize + trainsize].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['NUM_CRIMES'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_CRIMES'][trainsize:testsize + trainsize] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

1.15.2
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb44dd928d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/dnnEncodedTDMaxMinSndpWeak5', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Runnin

Temp Dewp  
Model 1 got a RMSE of 55.5397 | Using the average of 730.3567 it got an RMSE of 77.4605  
Model 2 got a RMSE of 54.0013 | Using the average of 730.5858 it got an RMSE of 79.2463  
Model 3 got a RMSE of 53.9647 | Using the average of 728.1052 it got an RMSE of 79.7763  
Model 4 got a RMSE of 51.8400 | Using the average of 729.0701 it got an RMSE of 76.7882  
Model 5 got an RMSE of 52.4623 | Using the average of 730.6104 it got an RMSE of 74.6401   
  

Temp Dewp Sndp  
Model 1 got a RMSE of 51.2643 | Using the average of 729.8836 it got an RMSE of 81.1761  
Model 2 got a RMSE of 63.0229 | Using the average of 729.8037 it got an RMSE of 78.2863  
Model 3 got a RMSE of 59.2860 | Using the average of 729.0918 it got an RMSE of 77.8681  
Model 4 got a RMSE of 50.3015 | Using the average of 730.3060 it got an RMSE of 78.7829  
Model 5 got an RMSE of 51.0692 | Using the average of 730.3537 it got an RMSE of 77.5745   

Temp Dewp Max Min  
Model 1 got a RMSE of 55.4920 | Using the average of 730.3201 it got an RMSE of 79.7384  
Model 2 got a RMSE of 51.9927 | Using the average of 731.2761 it got an RMSE of 79.4068  
Model 3 got a RMSE of 49.4655 | Using the average of 730.0179 it got an RMSE of 75.5120  
Model 4 got a RMSE of 52.6090 | Using the average of 729.7619 it got an RMSE of 71.7270  
Model 5 got an RMSE of 48.0962 | Using the average of 730.5246 it got an RMSE of 76.9214   

Temp Dewp Max Min Sndp  
Model 1 got a RMSE of 51.2402 | Using the average of 729.8403 it got an RMSE of 74.6624  
Model 2 got a RMSE of 53.1222 | Using the average of 729.9164 it got an RMSE of 77.9991  
Model 3 got a RMSE of 48.5441 | Using the average of 730.4299 it got an RMSE of 73.4904  
Model 4 got a RMSE of 51.6228 | Using the average of 731.2739 it got an RMSE of 76.9017  
Model 5 got an RMSE of 54.7547 | Using the average of 730.0575 it got an RMSE of 77.9729   

In [158]:
# Creating a variable for the last 5% of the data that is easy to handle
input = predictors[testsize + trainsize:]

# Different Data sets
# dnnEncodedTDWeak
# dnnEncodedTDSnowWeak
# dnnEncodedTDMaxMinWeak
# dnnEncodedTDMaxMinSndpWeak
# Boilerplate for creating the estimator
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/dnnEncodedTDMaxMinSndpWeak5', hidden_units=[20,18,14], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

# Predicting the values 
preds = estimator.predict(x=input.values)

# Creating a list of predictions 
predslist = preds['scores']*SCALE_NUM_TRIPS

# Creating the RMSE by comparing the targets to the list of predictions
rmse = np.sqrt(np.mean((targets[testsize + trainsize:].values - predslist)**2))

# Calculate the mean of the Number of Crimes Values.
avg = np.mean(shuffle['NUM_CRIMES'][testsize + trainsize:])

print()
# Printing the rmse and average crimes
print('DNNRegression has RMSE of {0}'.format(rmse));
print('Average is {0}'.format(avg))


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb4532bad10>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/dnnEncodedTDMaxMinSndpWeak5', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/dnnEncodedTDMaxMinSndpWeak5/model.ckpt-10000

Temp Dewp  
Model 1 got a RMSE of 64.3974  
Model 2 got a RMSE of 49.6896  
Model 3 got a RMSE of 46.5449  
Model 4 got a RMSE of 43.4516  
Model 5 got an RMSE of 55.1923  

Temp Dewp Sndp  
Model 1 got a RMSE of 52.1821  
Model 2 got a RMSE of 67.2452  
Model 3 got a RMSE of 55.0430  
Model 4 got a RMSE of 56.7479  
Model 5 got an RMSE of 54.6329   

Temp Dewp Max Min  
Model 1 got a RMSE of 46.8801  
Model 2 got a RMSE of 54.4286  
Model 3 got a RMSE of 54.8629  
Model 4 got a RMSE of 53.9686  
Model 5 got an RMSE of 56.5337   


Temp Dewp Max Min Sndp  
Model 1 got a RMSE of 49.8316  
Model 2 got a RMSE of 55.2565  
Model 3 got a RMSE of 59.1936  
Model 4 got a RMSE of 53.5936  
Model 5 got an RMSE of 44.2207   